## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-02-03-42-00 +0000,wsj,BlueScope Steel Open to Better Offers After Sp...,https://www.wsj.com/business/bluescope-steel-o...
1,2026-02-02-03-38-34 +0000,startribune,The latest: Musicians call out ICE actions in ...,https://www.startribune.com/ice-raids-minnesot...
2,2026-02-02-03-23-22 +0000,nypost,CBS News contributor and anti-aging physician ...,https://nypost.com/2026/02/01/us-news/cbs-news...
3,2026-02-02-03-22-57 +0000,bbc,Bad Bunny declares 'ICE out' as he wins Grammy,https://www.bbc.com/news/articles/ce8g7q4ymrvo...
4,2026-02-02-03-21-00 +0000,wsj,The Famous Names in the Latest Release of Epst...,https://www.wsj.com/us-news/law/what-we-know-s...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2408/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
44,trump,24
30,epstein,17
12,ice,14
70,iran,12
91,year,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
31,2026-02-02-00-53-49 +0000,nypost,Trump announces two-year closure of Trump Kenn...,https://nypost.com/2026/02/01/us-news/trump-an...,79
11,2026-02-02-02-56-44 +0000,nyt,Trump Says Kennedy Center Will Close for 2-Yea...,https://www.nytimes.com/2026/02/01/us/politics...,58
65,2026-02-01-22-34-48 +0000,nypost,India to buy oil from Venezuela rather than Ru...,https://nypost.com/2026/02/01/world-news/india...,57
14,2026-02-02-02-49-20 +0000,nypost,"5-year-old boy, father taken into custody by I...",https://nypost.com/2026/02/01/us-news/5-year-o...,55
138,2026-02-01-15-09-48 +0000,nypost,Trump jokes about suing new Fed chair nominee ...,https://nypost.com/2026/02/01/us-news/trump-jo...,55


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
31,79,2026-02-02-00-53-49 +0000,nypost,Trump announces two-year closure of Trump Kenn...,https://nypost.com/2026/02/01/us-news/trump-an...
152,46,2026-02-01-13-52-32 +0000,nypost,Ex-Prince Andrew’s creepy reply to Jeffrey Eps...,https://nypost.com/2026/02/01/world-news/ex-pr...
14,45,2026-02-02-02-49-20 +0000,nypost,"5-year-old boy, father taken into custody by I...",https://nypost.com/2026/02/01/us-news/5-year-o...
174,39,2026-02-01-10-54-14 +0000,nypost,Iran’s supreme leader warns any US attack woul...,https://nypost.com/2026/02/01/world-news/iran-...
24,36,2026-02-02-01-31-39 +0000,nypost,Mike Johnson ‘confident’ gov’t shutdown will e...,https://nypost.com/2026/02/01/us-news/mike-joh...
65,33,2026-02-01-22-34-48 +0000,nypost,India to buy oil from Venezuela rather than Ru...,https://nypost.com/2026/02/01/world-news/india...
45,32,2026-02-02-00-07-50 +0000,nypost,Bizarre origins of Gavin Newom’s ‘look’: Gover...,https://nypost.com/2026/02/01/us-news/gavin-ne...
1,24,2026-02-02-03-38-34 +0000,startribune,The latest: Musicians call out ICE actions in ...,https://www.startribune.com/ice-raids-minnesot...
59,24,2026-02-01-23-32-35 +0000,nypost,"Three clues may break mystery of woman, 20, wh...",https://nypost.com/2026/02/01/us-news/three-cl...
41,23,2026-02-02-00-11-29 +0000,nypost,Norway’s Crown Princess Mette-Marit sent Epste...,https://nypost.com/2026/02/01/world-news/norwa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
